In [1]:
#export
"""
This includes helper clis that make it quick to graph graphviz plots."""
__all__ = ["markers", "Markers", "map3d"]
import re, k1lib, math, os, numpy as np, io, json, base64, html
from k1lib.cli.init import BaseCli; import k1lib.cli as cli, k1lib.cli.init as init
cv2 = k1lib.dep("cv2", "opencv-python", "https://opencv.org/")
aruco = k1lib.dep("cv2.aruco", "opencv-python", "https://opencv.org/")
from collections import deque, defaultdict
settings = k1lib.settings.cli

/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [14]:
init.patchNumpy()

In [19]:
#export
class markers(BaseCli):
    def __init__(self, d:str=None):
        """Detect aruco markers. Returns :class:`Markers`.
Example::

    im = "some/image.jpg" | toImg()
    markers = im | kcv.markers()
    
    markers # run this in a notebook cell to quickly view where the bounding boxes are
    markers | item() # grab first detection, a tuple (aruco dictionary, coords (with shape (4,2)), number)
    markers[0] # same as above
    markers | deref() # dereference into just lists, instead of kcv.Markers object

For a moderately complex scene with 20 markers of size (1000, 800), this should take ~15ms to run. The same
scene of size (4000, 3000) should take ~170ms to run

:param d: aruco dictionary string, like 'DICT_7X7_1000'. If specified, then will only look in
    those dictionaries. If not specified will look in all dictionaries"""
        self.ds = [d] if d else ["DICT_4X4_1000", "DICT_5X5_1000", "DICT_6X6_1000", "DICT_7X7_1000"]
    def __ror__(self, im):
        arr = im | cli.toNdArray(np.uint8) | cli.op().transpose(1, 2, 0)
        return Markers(im, self.ds | cli.apply(lambda d: [d, cv2.aruco.detectMarkers(arr, aruco.Dictionary_get(getattr(aruco, d)))[:2]]) | cli.deref() | ~cli.filt("x[1] is None", 1) | cli.apply(cli.item().all(2) | cli.T() , 1) | cli.ungroup(False) | cli.deref())
class Markers:
    def __init__(self, im, ms):
        """Resulting Markers object, obtained from :class:`markers` cli, not
intended to be instantiated by the end user."""
        self.im = im; self._ms = ms
    def __getitem__(self, s): return self._ms[s]
    def __iter__(self): return iter(self._ms)
    def __len__(self): return len(self._ms)
    def __repr__(self): return f"<Markers n={len(self)} im.shape={self.im | cli.shape() | cli.op()[::-1]} ds={self._ms | cli.cut(0) | cli.count() | ~cli.sort() | cli.cut(1) | cli.deref()}>"
    def _repr_html_(self):
        im = self.im; ms = self._ms; p5 = k1lib.p5; p5.newSketch(*im | cli.shape(), False)
        p5.image(im, 0, 0); p5.background(200, 220)
        for _dict, coords, num in ms:
            a,b,c,d = coords
            p5.stroke(255, 0, 0);   p5.line(*a,*b)
            p5.stroke(0, 255, 0);   p5.line(*b,*c)
            p5.stroke(0, 0, 255);   p5.line(*c,*d)
            p5.stroke(255, 255, 0); p5.line(*d,*a)
            p5.stroke(255, 0, 255); p5.text(f"{num}", *a)
        return f"{html.escape(self.__repr__())}<br>{p5.svg()}"

In [29]:
%%time
im = '~/repos/labs/mlexps/opencv/3-charuco/store/PXL_20240111_160750752.jpg' | cli.toImg()# | cli.aS(tf.Resize(800))
assert im | markers("DICT_6X6_1000") | cli.shape() == (20, 3, 13)

CPU times: user 474 ms, sys: 19.2 ms, total: 493 ms
Wall time: 250 ms


In [113]:
#export
def map3d(ws,cs):
    """Creates 2 functions that maps from world points (ws) to camera points (cs).
Example::

    ws = np.random.randn(10, 3)
    cs = np.copy(ws); cs[:,1] = -cs[:,1]; cs[:,2] = -cs[:,2]
    
    f1, f2 = kcv.map3d(ws, cs)
    
    f1(ws) # returns matrix similar to cs
    f2(cs) # returns matrix similar to ws
"""
    wc = np.mean(ws, axis=0); wsp = ws - wc
    cc = np.mean(cs, axis=0); csp = cs - cc
    H = wsp.T @ csp; U, S, Vt = np.linalg.svd(H)
    R = np.dot(U, Vt); R_inv = np.linalg.inv(R)
    t = cc - np.dot(R, wc)
    # main equation is given by chatgpt, but the transformation for some reason is translated by a
    # fixed vector, so this is to figure out the correct bias vector to add to
    b1 = cc-(ws@R+t) | cli.T() | cli.toMean().all()
    b2 = ws-(cs-t)@R_inv | cli.T() | cli.toMean().all()
    return lambda vs: (vs)@R+t+b1, lambda vs: (vs-t)@R_inv+b2

In [114]:
ws = np.random.randn(10, 3)
cs = np.copy(ws); cs[:,1] = -cs[:,1]; cs[:,2] = -cs[:,2]
f1, f2 = map3d(ws, cs)
assert abs(f1(ws) - cs).sum() < 1e-6
assert abs(f2(cs) - ws).sum() < 1e-6

In [112]:
!../../export.py cli/kcv --upload=True

/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2024-01-17 14:29:13,442	INFO worker.py:1458 -- Connecting to existing Ray cluster at address: 192.168.1.19:6379...
2024-01-17 14:29:13,450	INFO worker.py:1633 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 
----- exportAll
13885   0   60%   
9205    1   40%   
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 1.5
Uninstalling k1lib-1.5:
  Successfully uninstalled k1lib-1.5
running install
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/si

In [1]:
!../../export.py cli/kcv

2024-03-03 03:23:20,332	INFO worker.py:1458 -- Connecting to existing Ray cluster at address: 192.168.1.19:6379...
2024-03-03 03:23:20,343	INFO worker.py:1633 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 
./export started up - /home/kelvin/anaconda3/envs/ray2/bin/python3
----- exportAll
15219   0   61%   
9780    1   39%   
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 1.5.2
Uninstalling k1lib-1.5.2:
  Successfully uninstalled k1lib-1.5.2
running install
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/easy_install.py:144: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
running bdi

In [34]:
!../../export.py cli/kcv --boostrap=True

/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2024-01-16 16:37:19,424	INFO worker.py:1458 -- Connecting to existing Ray cluster at address: 192.168.1.19:6379...
2024-01-16 16:37:19,433	INFO worker.py:1633 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 
----- exportAll
13833   0   60%   
9165    1   40%   
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 1.5
Uninstalling k1lib-1.5:
  Successfully uninstalled k1lib-1.5
running install
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/si